In [1]:
!pip install pyarrow

In [2]:
import pandas as pd

df_jan = pd.read_parquet("yellow_tripdata_2023-01.parquet")
df_feb = pd.read_parquet("yellow_tripdata_2023-02.parquet")
df_feb.shape[1]

19

In [3]:
len(df_jan.columns)

19

In [4]:
df_jan.columns == df_feb.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False])

In [5]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [6]:
df_feb.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [7]:
df_jan["tpep_pickup_datetime"].isna().sum()

0

In [8]:
df_jan["tpep_dropoff_datetime"].isna().sum()

0

In [9]:
duration_all = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
duration_all.std()

Timedelta('0 days 00:42:35.661074')

In [10]:
filtered_df = df_jan[
    (
        (df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"])
        >= pd.Timedelta(minutes=1)
    )
    & (
        (df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"])
        <= pd.Timedelta(minutes=60)
    )
]

filtered_df_val = df_feb[
    (
        (df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"])
        >= pd.Timedelta(minutes=1)
    )
    & (
        (df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"])
        <= pd.Timedelta(minutes=60)
    )
]

In [11]:
duration = filtered_df['tpep_dropoff_datetime'] - filtered_df['tpep_pickup_datetime']

In [12]:
duration

0         0 days 00:08:26
1         0 days 00:06:19
2         0 days 00:12:45
3         0 days 00:09:37
4         0 days 00:10:50
                ...      
3066761   0 days 00:13:59
3066762   0 days 00:19:27
3066763   0 days 00:24:31
3066764   0 days 00:13:00
3066765   0 days 00:14:24
Length: 3009173, dtype: timedelta64[us]

In [13]:
duration.std()

Timedelta('0 days 00:09:56.363137')

In [14]:
len(duration)/len(duration_all)

0.9812202822125979

In [15]:
filtered_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [16]:
from sklearn.feature_extraction import DictVectorizer

# convert location IDs to strings
filtered_df = filtered_df.assign(PULocationID=filtered_df['PULocationID'].astype(str))
filtered_df = filtered_df.assign(DOLocationID=filtered_df['DOLocationID'].astype(str))
filtered_df_val = filtered_df_val.assign(PULocationID=filtered_df_val['PULocationID'].astype(str))
filtered_df_val = filtered_df_val.assign(DOLocationID=filtered_df_val['DOLocationID'].astype(str))

: 

In [ ]:
data_dicts = filtered_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
data_dicts_val = filtered_df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [ ]:
data_dicts

In [ ]:
vectorizer = DictVectorizer(sparse=True)
feature_matrix = vectorizer.fit_transform(data_dicts)

In [ ]:
print("Feature names:", vectorizer.get_feature_names_out())

In [ ]:
feature_matrix.shape[1]

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
target = duration.dt.total_seconds() / 60.
target

In [ ]:
model = LinearRegression()
model.fit(feature_matrix, target)

In [ ]:
predictions = model.predict(feature_matrix)
predictions

In [ ]:
# Calculate RMSE on train
rmse = np.sqrt(mean_squared_error(target, predictions))
print("RMSE on the training data:", rmse)

In [ ]:
filtered_df_val = df_feb[
    (
        (df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"])
        >= pd.Timedelta(minutes=1)
    )
    & (
        (df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"])
        <= pd.Timedelta(minutes=60)
    )
]

duration_validation = filtered_df_val['tpep_dropoff_datetime'] - filtered_df_val['tpep_pickup_datetime']
# convert location IDs to strings
filtered_df_val['PULocationID'] = filtered_df_val['PULocationID'].astype(str)
filtered_df_val['DOLocationID'] = filtered_df_val['DOLocationID'].astype(str)


In [ ]:
data_dicts_validation = filtered_df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')


In [ ]:
# only fit DO NOT transform to CV set 
feature_matrix_validation = vectorizer.fit(data_dicts_validation)
target_validation = duration_validation.dt.total_seconds() / 60.

In [ ]:
predictions_validation = model.predict(feature_matrix_validation)
predictions_validation

In [ ]:
# Calculate RMSE on validation
rmse_validation = np.sqrt(mean_squared_error(target, predictions_validation))
print("RMSE on the training data:", rmse)